In [ ]:
!pip install pandas
!pip install sqlalchemy
!pip install psycopg2

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# credebtuaks are arbitraty to local container so safe to put inline
engine = create_engine("postgresql://postgres:postgres@pg_taxi_data:5432/postgres")

In [ ]:
def execute_statement(statement, engine):
    from pprint import pprint as pprint
    with engine.connect() as conn:
        curr = conn.execute(statement)
        for row in curr:
            pprint(row)

In [ ]:
# fetching column names for ease of working
pd.read_sql("SELECT * FROM taxi_trips LIMIT 1", con=engine)

In [ ]:
pd.read_sql("SELECT * FROM taxi_zones LIMIT 1", con=engine)

In [ ]:
# testing connection
execute_statement("SELECT count(1) FROM taxi_trips", engine)

In [ ]:
# Question 3: How many trips were totally made on January 15
# Methodology: the trip has to start and end on the same date
# Approach: cast both timestamps to date (postgres understands YYYY-MM-DD)
execute_statement("SELECT count(1) FROM taxi_trips WHERE lpep_pickup_datetime::date = '2019-01-15' and lpep_dropoff_datetime::date = '2019-01-15'", engine)

In [ ]:
# Question 4: Which was the day with the largest trip distance
# Methodology: fetch date with max travel distance
# Approach: there are two possible approaches here. The more consistent one that requires more computational power
# is sorting. Another one would be selecting map trip distance first and then getting the date for it
# 2 fetches
execute_statement("SELECT lpep_pickup_datetime, lpep_dropoff_datetime FROM taxi_trips WHERE trip_distance = (SELECT max(trip_distance) from taxi_trips)", engine)
# sorting
execute_statement("SELECT lpep_pickup_datetime, lpep_dropoff_datetime FROM taxi_trips ORDER BY trip_distance DESC LIMIT 1", engine)

In [ ]:
# Question 5: In 2019-01-01 how many trips had 2 and 3 passengers?
# Methodology: since fetching the same dates (start and end) returns result that is not in the answers, explore start and end separately
# Approach: filter by passenger count and date, fetch count of start and end dates to make sure we are consistent
print("Start & End")
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_pickup_datetime::date = '2019-01-01' and lpep_dropoff_datetime::date = '2019-01-01' and passenger_count = 2", engine)
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_pickup_datetime::date = '2019-01-01' and lpep_dropoff_datetime::date = '2019-01-01' and passenger_count = 3", engine)

print("Start")
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_pickup_datetime::date = '2019-01-01' and passenger_count = 2", engine)
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_pickup_datetime::date = '2019-01-01' and passenger_count = 3", engine)

print("End")
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_dropoff_datetime::date = '2019-01-01' and passenger_count = 2", engine)
execute_statement("SELECT count(lpep_pickup_datetime), count(lpep_dropoff_datetime) FROM taxi_trips WHERE lpep_dropoff_datetime::date = '2019-01-01' and passenger_count = 3", engine)

In [ ]:
# Question 6: For the passengers picked up in the Astoria Zone which was the drop up zone that had the largest tip?
# Methodology: we could sql join the two tables, or take a multi-step approach. Since the column names are capitalized, we take it in ""
# Approach: 
# Step 1: Fetch an ID for Astoria Zone
# Step 2: Fetch largest tip for Astoria Zone ID and drop off zone ID
# Step 3: Fetch drop off zone ID name

# Stepwise
# execute_statement("""SELECT "LocationID" FROM taxi_zones WHERE "Zone" = 'Astoria'""", engine) # returns 7
# execute_statement("""SELECT max(tip_amount) FROM taxi_trips WHERE "PULocationID" = 7""", engine) # returns 88
# execute_statement("""SELECT "DOLocationID" FROM taxi_trips WHERE tip_amount = 88.0""", engine) # returns 146
# execute_statement("""SELECT "Zone" FROM taxi_zones WHERE "LocationID" = 146""", engine)

# Single query using subqueries
query = """
    SELECT "Zone"
    FROM (
        SELECT "DOLocationID", tip_amount
        FROM (
            SELECT tip_amount, "PULocationID", "Zone", "DOLocationID"
            FROM taxi_trips tt
            LEFT JOIN taxi_zones tz
            ON "PULocationID" = "LocationID"
            ) as pull_join
        WHERE "Zone" = 'Astoria'
        ORDER BY tip_amount DESC LIMIT 1
        ) top_tip_in_astoria
    JOIN taxi_zones
    ON "DOLocationID" = "LocationID"
        """

execute_statement(query, engine)
